In [7]:
conda install openpyxl

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import datetime as dt
import numpy as np
import glob
import openpyxl

In [76]:
archivosAll = np.array(glob.glob('../data/Eólica 1/all_files/*'))
rutas = pd.DataFrame({'files': archivosAll})
rutas

,files
0,../data/Eólica 1/all_files/E1_22-10-2021.csv
1,../data/Eólica 1/all_files/E1_20-04-22.csv
2,../data/Eólica 1/all_files/20682654.xlsx
3,../data/Eólica 1/all_files/E1_17-03-2022.csv
4,../data/Eólica 1/all_files/E1_29-03-2022.csv
5,../data/Eólica 1/all_files/E1_17-03-2022.xlsx
6,../data/Eólica 1/all_files/E1_14-12-2021.csv
7,../data/Eólica 1/all_files/20682650.csv
8,../data/Eólica 1/all_files/20682654.csv
9,../data/Eólica 1/all_files/E1_22-10-2021.xlsx


In [90]:
rutas.files.str.contains('csv').values

array([ True,  True, False,  True,  True, False,  True,  True,  True,
       False, False, False, False, False])

In [118]:
eolica1 = []
for index, value in rutas.files.str.contains('csv').items():
    if (value== True):
        #print(rutas.files[index])
        file_readed = pd.read_csv(rutas.files[index], skiprows=1, usecols=[1,2,3,4]).set_axis(['fecha','Vel_kmh', 'Vel_Raf_kmh', 'Dir_viento_grd'], axis='columns')
        eolica1.append(file_readed)
        #print(file_readed)
    else:
        #print(rutas.files[index])
        file_readed = pd.read_excel(rutas.files[index], skiprows=1, usecols=[1,2,3,4]).set_axis(['fecha','Vel_kmh', 'Vel_Raf_kmh', 'Dir_viento_grd'], axis='columns')
        eolica1.append(file_readed)
        #print(file_readed)

pd.concat(eolica1).reset_index(drop=True)
        

,fecha,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd
0,06/08/21 04:34:05 PM,28.872,42.084,206.4
1,06/08/21 04:35:05 PM,32.472,39.672,210.6
2,06/08/21 04:36:05 PM,27.648,36.072,226.0
3,06/08/21 04:37:05 PM,30.06,38.484,219.0
4,06/08/21 04:38:05 PM,24.048,27.648,219.0
...,...,...,...,...
466197,12/14/21 07:55:59 AM,3600.0,6012.000,263.9
466198,12/14/21 07:56:59 AM,3600.0,4824.000,262.5
466199,12/14/21 07:57:59 AM,2412.0,3600.000,261.1
466200,12/14/21 07:58:59 AM,2412.0,3600.000,261.1


In [9]:
archivos = np.array(glob.glob('../data/Eólica 1/csv' + "/*.csv"))
print(archivos)
print(archivos.shape)

['../data/Eólica 1/csv/E1_22-10-2021.csv'
 '../data/Eólica 1/csv/E1_20-04-22.csv'
 '../data/Eólica 1/csv/E1_17-03-2022.csv'
 '../data/Eólica 1/csv/E1_29-03-2022.csv'
 '../data/Eólica 1/csv/E1_14-12-2021.csv'
 '../data/Eólica 1/csv/20682650.csv' '../data/Eólica 1/csv/20682654.csv']
(7,)


In [55]:
def lectura(file):
    file_readed = pd.read_csv(file, skiprows=1, usecols=[1,2,3,4]).set_axis(['fecha','Vel_kmh', 'Vel_Raf_kmh', 'Dir_viento_grd'], axis='columns')
    return file_readed

In [56]:
data1 = []
for file in archivos:
    #print(file)
    temp = lectura(file)
    #print(temp)
    data1.append(temp)
    
eolica1_csv = pd.concat(data1, ignore_index=True)
eolica1_csv


,fecha,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd
0,06/08/21 04:34:05 PM,28.872,42.084,206.4
1,06/08/21 04:35:05 PM,32.472,39.672,210.6
2,06/08/21 04:36:05 PM,27.648,36.072,226.0
3,06/08/21 04:37:05 PM,30.060,38.484,219.0
4,06/08/21 04:38:05 PM,24.048,27.648,219.0
...,...,...,...,...
266112,04/23/20 03:10:00 PM,32.472,36.072,338.3
266113,04/23/20 03:11:00 PM,32.472,40.860,336.9
266114,04/23/20 03:12:00 PM,31.248,39.672,318.7
266115,04/23/20 03:13:00 PM,36.072,45.684,300.4


In [38]:
pd.to_datetime('13000101', format='%Y%m%d', errors='ignore')

datetime.datetime(1300, 1, 1, 0, 0)

In [42]:
#dt.strptime(eolica1_csv['fecha'][0], '%d/%M/%y  %I:%M:%S %p') 

pd.to_datetime(eolica1_csv['fecha'])
 

0        2021-06-08 16:34:05
1        2021-06-08 16:35:05
2        2021-06-08 16:36:05
3        2021-06-08 16:37:05
4        2021-06-08 16:38:05
                 ...        
266112   2020-04-23 15:10:00
266113   2020-04-23 15:11:00
266114   2020-04-23 15:12:00
266115   2020-04-23 15:13:00
266116   2019-11-03 10:12:28
Name: fecha, Length: 266117, dtype: datetime64[ns]

In [14]:
archivos_xlsx = np.array(glob.glob('../data/Eólica 1/xlsx' + "/*.xlsx"))
print(archivos_xlsx)
print(archivos_xlsx.shape)

['../data/Eólica 1/xlsx/20682654.xlsx'
 '../data/Eólica 1/xlsx/E1_17-03-2022.xlsx'
 '../data/Eólica 1/xlsx/E1_22-10-2021.xlsx'
 '../data/Eólica 1/xlsx/E1_14-07-22.xlsx'
 '../data/Eólica 1/xlsx/20682650.xlsx'
 '../data/Eólica 1/xlsx/E1 19-11-19.xlsx'
 '../data/Eólica 1/xlsx/E1_14-12-2021.xlsx']
(7,)


In [12]:
def lectura_xslx(file):
    file_readed = pd.read_excel(file, skiprows=2, usecols=[1,2,3,4]).set_axis(['fecha','Vel_kmh', 'Vel_Raf_kmh', 'Dir_viento_grd'], axis='columns')
    return file_readed

In [22]:
lectura_xslx(archivos_xlsx[4])

,fecha,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd
0,03/13/20 08:53:00 AM,0.0,0.0,349.6
1,03/13/20 08:54:00 AM,0.0,0.0,349.6
2,03/13/20 08:55:00 AM,0.0,1188.0,349.6
3,03/13/20 08:56:00 AM,1188.0,1188.0,351.0
4,03/13/20 08:57:00 AM,0.0,1188.0,351.0
...,...,...,...,...
59416,04/23/20 03:09:00 PM,27648.0,34848.0,313.1
59417,04/23/20 03:10:00 PM,32472.0,36072.0,338.3
59418,04/23/20 03:11:00 PM,32472.0,40860.0,336.9
59419,04/23/20 03:12:00 PM,31248.0,39672.0,318.7


In [51]:
data2 = []
for file in archivos_xlsx:
    #print(file)
    temp = lectura_xslx(file)
    #print(temp)
    data2.append(temp)
    
eolica1_xlsx = pd.concat(data2, ignore_index=True)
eolica1_xlsx

,fecha,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd
0,2022-07-01 08:11:37,10836.0,12024.0,251.3
1,2022-07-01 08:12:37,12024.0,13212.0,241.5
2,2022-07-01 08:13:37,10836.0,13212.0,248.5
3,2022-07-01 08:14:37,13212.0,15624.0,249.9
4,2022-07-01 08:15:37,12024.0,13212.0,244.3
...,...,...,...,...
200080,12/14/21 07:55:59 AM,3600.0,6012.0,263.9
200081,12/14/21 07:56:59 AM,3600.0,4824.0,262.5
200082,12/14/21 07:57:59 AM,2412.0,3600.0,261.1
200083,12/14/21 07:58:59 AM,2412.0,3600.0,261.1
